In [1]:
#!git clone https://github.com/starbucksdolcelatte/ShowMeTheColor

In [12]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense,Flatten,Input
import os
from tensorflow.keras.callbacks import Callback

In [13]:
size=(224,224,3)
size

(224, 224, 3)

In [14]:
image = ImageDataGenerator(
    rescale=1/255,
    rotation_range=30,       # 회전 범위 확대
    width_shift_range=0.3,   # 수평 이동 확대
    height_shift_range=0.3,  # 수직 이동 확대
    shear_range=0.3,         # 전단 범위 확대
    zoom_range=0.3,          # 줌 범위 확대
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # 밝기 조정
    fill_mode="nearest"
)


In [15]:
train=image.flow_from_directory(os.path.join('img','train'),target_size=size[:2],batch_size=32,shuffle=True)
test=image.flow_from_directory(os.path.join('img','test'),target_size=size[:2],batch_size=32,shuffle=True)

Found 213 images belonging to 4 classes.


Found 40 images belonging to 4 classes.


In [16]:
mobilenet=VGG16(include_top=False,input_shape=size)
mobilenet.summary()

for layer in mobilenet.layers:
    layer.trainable = False

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model=Sequential((Input(shape=size),mobilenet,Flatten(),Dense(64,activation="relu"),Dense(len(train.class_indices),activation="softmax")))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,320,644 (62.26 MB)

 Trainable params: 1,605,956 (6.13 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [18]:
train.class_indices

{'fall': 0, 'spring': 1, 'summer': 2, 'winter': 3}

In [19]:
with open("labels.txt",'w') as f:
    f.write("\n".join(['가을 웜톤','봄 웜톤','여름 쿨톤','겨울 쿨톤']))    

In [20]:
class TerminateOnThreshold(Callback):
    def __init__(self, accuracy,val_accuracy):
        super(TerminateOnThreshold, self).__init__()
        self.accuracy = accuracy
        self.val_accuracy = val_accuracy

    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get("val_accuracy")  # 정확도 확인
        accuracy = logs.get("accuracy") 
        if accuracy and val_accuracy and accuracy >= self.accuracy and val_accuracy >= self.val_accuracy:
            print(f"\nEpoch {epoch+1}: Reached accuracy {accuracy:.4f} (>= {self.accuracy}), val_accuracy {val_accuracy:.4f} (>= {self.val_accuracy}), stopping training!")
            self.model.stop_training = True

# 사용자 정의 콜백 설정
terminate_on_threshold = TerminateOnThreshold(accuracy=0.9,val_accuracy=0.8)
model.fit(train, epochs=999999, validation_data=test, callbacks=[terminate_on_threshold])


Epoch 1/999999
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.2666 - loss: 3.2550 - val_accuracy: 0.2500 - val_loss: 1.6662
Epoch 2/999999
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.2264 - loss: 1.6581 - val_accuracy: 0.2500 - val_loss: 1.5038
Epoch 3/999999
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.2221 - loss: 1.5546 - val_accuracy: 0.2250 - val_loss: 1.4816
Epoch 4/999999
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.3256 - loss: 1.4139 - val_accuracy: 0.2750 - val_loss: 1.4084
Epoch 5/999999
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.2592 - loss: 1.4019 - val_accuracy: 0.3750 - val_loss: 1.3665
Epoch 6/999999
7/7 ━━━━━━━━━━━━━━━━━━━━ 35s 5s/step - accuracy: 0.2898 - loss: 1.3850 - val_accuracy: 0.3000 - val_loss: 1.4242
Epoch 7/999999
7/7 ━━━━━━━━━━━━━━━━━━━━ 35s 5s/step - accuracy: 0.3529 - loss: 1.3474 - val_accuracy: 0.3250 - val_loss: 1.3873
Epoch 8/999999
7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 5s/step - accuracy: 0.3348 - loss: 1.3676 - val_accuracy: 0.

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(test)
print(f"Test Accuracy: {(accuracy)*100}")

In [ ]:
model.save('model.keras')